In [ ]:
!pip install emoji 
!pip install transformers
!pip install itranslate
!pip install wget
!pip install liwc

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import cv2
import pandas as pd
import tensorflow as tf
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import transformers
from itranslate import itranslate as itrans
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
import emoji
import nltk
from nltk.corpus import stopwords
import os
import wget
##Descarga de las stopwords por NLTK
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
##Función para quitar link, hashtags, emojis y caracteres especiales.
def clean_text(text):
    clean=text
    reg = re.compile('\&amp')
    clean = clean.apply(lambda r: re.sub(reg, string=r, repl='&'))
    reg = re.compile('\\n')
    clean = clean.apply(lambda r: re.sub(reg, string=r, repl=' '))
    reg = re.compile('@[a-zA-Z0-9\_]+')
    clean = clean.apply(lambda r: re.sub(reg, string=r, repl='username'))
    reg = re.compile('#[a-zA-Z0-9\_]+')
    clean = clean.apply(lambda r: re.sub(reg, string=r, repl='#'))
    reg = re.compile('https?\S+(?=\s|$)')
    clean = clean.apply(lambda r: re.sub(reg, string=r, repl='url'))
    reg = re.compile('[0-9\_]+')
    clean = clean.apply(lambda r: re.sub(reg, string=r, repl='0'))
    clean = clean.replace(r'[^\x00-\x7F]+', '', regex=True)
    return clean.apply(lambda x: ' '.join(re.sub("(#[A-Za-z0-9]+)|(/[A-Za-z0-9]+)|(@[A-Za-z0-9]+)|(www.[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|(/+:\/\/\S+)"," ",x).split()))


#Función para remover las stopwords
def byestop(text):
    nostop = text
    stop = stopwords.words('english')
    ##algo raro pasa que se tenía que pasar dos veces por el filtro para que se removieran las palabras, luego lo checo.
    nostop = text.apply(lambda words: ' '.join(word.lower() for word in words.split() if word not in stop))
    return nostop.apply(lambda words: ' '.join(word.lower() for word in words.split() if word not in stop))

#Función para tradicir textos
def trad(text):
    return text.apply(lambda x: itrans(x, to_lang="en"))

#Función para reducir la calidad y tamaño de las imagenes
def imgnorm(paths):
    images = paths.tolist()
    for image in images:
      img = cv2.imread(image)
      img = cv2.resize(img,(100,100))
      cv2.imwrite(image, img)
      img = cv2.resize(img,(400,400))
      cv2.imwrite(image, img)

#Función para obtener la lista de features de las imagenes
def to_image_features(path):
  hsv_list = []
  img = io.imread(path)
  img = color.rgb2hsv(img)
  hsv = []
  for i in range(len(img.shape)):
      if len(img.shape) == 2:
          imdata = img[:, i]
      elif len(img.shape) == 3:
          imdata = img[:, :, i]
      avg = np.mean(imdata)
      hsv.append(avg)
  hsv_list.append(hsv)
  return np.mean(hsv_list, axis=0)

In [ ]:
##Cargamos el dataframe
df = pd.read_csv("instagram-posts.csv")
df

,image,caption,depression
0,https://instagram.fntr6-1.fna.fbcdn.net/v/t51....,⇒ follow me @yeasad for more edits 💜,1
1,https://instagram.fntr6-3.fna.fbcdn.net/v/t51....,😢😢😢😞....#cry #crying #cryingquotes #cryingquot...,1
2,https://instagram.fntr6-1.fna.fbcdn.net/v/t51....,NEVER! 🥀🥲Please like and save! 🥺Follow @deep_....,1
3,https://instagram.fntr6-3.fna.fbcdn.net/v/t51....,Ever 🥺❤️..If viewing follow @deepvibes_._ for ...,1
4,https://instagram.fntr6-3.fna.fbcdn.net/v/t51....,💞💞💞Follow @heartinntears for more quotes#brok...,1
...,...,...,...
1995,https://scontent-frx5-2.cdninstagram.com/v/t51...,#hijab #hijabhits #hijabers #hijabmurah #hijab...,0
1996,https://scontent-frt3-1.cdninstagram.com/v/t51...,100 Tagú Cigányzenekar Koncert#Vajdahunyadvar#...,0
1997,https://scontent-frt3-2.cdninstagram.com/v/t51...,Dag 109 med #positivevibes ♥️ Har lånt dette s...,0
1998,https://scontent-frx5-1.cdninstagram.com/v/t51...,Who else is excited about the weekend?🤗🤗🤗🤗Here...,0


In [ ]:
#Limpiamos y traducimos los datos de las captions
df.caption = clean_text(df.caption)
df.caption = trad(df.caption)
df = df[df.caption.str.strip().str.len() != 0]
#df.to_csv("semipreproc.csv")

[E 210731 02:47:51 itranslate:135] 429 Client Error: Too Many Requests for url: https://www.google.com/sorry/index?continue=https://translate.google.cn/_/TranslateWebserverUi/data/batchexecute&q=EgQiS34sGNbzkogGIhBcvYbiSjlSM3AcJfnQZFDhMgFy
    For more information check: https://httpstatuses.com/429


HTTPStatusError: ignored

In [ ]:
df

,image,caption,depression
0,https://instagram.fntr6-1.fna.fbcdn.net/v/t51....,follow me username for more edits,1
2,https://instagram.fntr6-1.fna.fbcdn.net/v/t51....,NEVER Please like and save Follow username quo...,1
3,https://instagram.fntr6-3.fna.fbcdn.net/v/t51....,Ever If viewing follow username 0 for more,1
4,https://instagram.fntr6-3.fna.fbcdn.net/v/t51....,Follow username for more quotes,1
5,https://instagram.fntr6-1.fna.fbcdn.net/v/t51....,Sadness from today,1
...,...,...,...
1994,https://scontent-frt3-2.cdninstagram.com/v/t51...,SKN day The gym walk now HNG with riches,0
1996,https://scontent-frt3-1.cdninstagram.com/v/t51...,0 Tagu Ciganyzenkar concert,0
1997,https://scontent-frt3-2.cdninstagram.com/v/t51...,Day 0 With lnt this super beautiful text s thi...,0
1998,https://scontent-frx5-1.cdninstagram.com/v/t51...,Who else is excited about the weekend Here s u...,0


In [ ]:
df.caption = byestop(df.caption)
#df.to_csv("semipreproc.csv")

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [ ]:
#df = pd.read_csv('preproc.csv')
#Print number of observations and features
print('This dataset has {} observations with {} features.'.format(df.shape[0], df.shape[1]))
#Check for null values
df.info()

This dataset has 1717 observations with 3 features.
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1717 entries, 0 to 1999
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   image       1717 non-null   object
 1   caption     1717 non-null   object
 2   depression  1717 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 53.7+ KB


In [ ]:
df

,image,caption,depression
0,https://instagram.fntr6-1.fna.fbcdn.net/v/t51....,follow username edits,1
2,https://instagram.fntr6-1.fna.fbcdn.net/v/t51....,never please like save follow username quotes0,1
3,https://instagram.fntr6-3.fna.fbcdn.net/v/t51....,ever viewing follow username 0,1
4,https://instagram.fntr6-3.fna.fbcdn.net/v/t51....,follow username quotes,1
5,https://instagram.fntr6-1.fna.fbcdn.net/v/t51....,sadness today,1
...,...,...,...
1994,https://scontent-frt3-2.cdninstagram.com/v/t51...,skn day gym walk hng riches,0
1996,https://scontent-frt3-1.cdninstagram.com/v/t51...,0 tagu ciganyzenkar concert,0
1997,https://scontent-frt3-2.cdninstagram.com/v/t51...,day 0 lnt super beautiful text thinking day al...,0
1998,https://scontent-frx5-1.cdninstagram.com/v/t51...,else excited weekend us saying happy weekend e...,0


In [ ]:
#images = df.image.head(3).tolist()
#path = "/content/drive/MyDrive/csv/img"
#counter = 0
#for image in images:
   #save_as = os.path.join(path, "image" + str(counter) + '.jpg')
    #wget.download(image, save_as)
    #images[counter] = save_as
    #counter += 1

In [ ]:
#dfs = pd.DataFrame(images)
#dfs.columns = ['a']
#dfs

,a
0,/content/drive/MyDrive/csv/img/image0.jpg
1,/content/drive/MyDrive/csv/img/image1.jpg
2,/content/drive/MyDrive/csv/img/image2.jpg


In [ ]:
import h5py
import os
import numpy as np
import pandas as pd
import liwc
from skimage import io, color
from typing import *
from collections import Counter
from pathlib import Path

In [ ]:
#Obtener las features de las imagenes para luego concatenar al vector de embedding
df['image_features'] = df['image'].apply(to_image_features)
df.head()

KeyError: ignored

In [ ]:
#Deshacerse de la columna de image
df.drop(["image"], axis=1, inplace=True)
df.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,caption,depression
0,follow username edits,1
2,never please like save follow username quotes0,1
3,ever viewing follow username 0,1
4,follow username quotes,1
5,sadness today,1


In [ ]:
#Definir los posts de depresion y de no depresión
depression_posts = df[df['depression'] == 1]
positive_posts = df[df['depression'] == 0]


depression_posts = depression_posts.loc[np.random.choice(depression_posts.index, 10, replace=False)]
positive_posts = positive_posts.loc[np.random.choice(positive_posts.index, 10, replace=False)]

In [ ]:
frames = [depression_posts, positive_posts]
df = pd.concat(frames)

In [ ]:
#Obtener el set de training y test, y hacer un shuffle
from sklearn.model_selection import train_test_split

train, test  = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
train.head()

,caption,depression,image_features
42,follow edits,1,"[0.5200924662030549, 0.3194566230166683, 0.763..."
53,fault stars along harsh realities life sufferi...,1,"[0.6705341902551907, 0.9672135259674282, 0.290..."
4,snuck hour noahs handsome daddy zombies well f...,0,"[0.2095022341691568, 0.5575005211596403, 0.542..."
70,follow like share tag someone follow support,1,"[0.09117565684253455, 0.03104956897475146, 0.2..."
1,dont time youre making priority,0,"[0.20479229127451606, 0.3114212465613828, 0.76..."


In [ ]:
test.head()

,caption,depression,image_features
55,comment agree follow alnfollow aln turn post n...,1,"[0.3230754736870737, 0.30060653253789654, 0.13..."
37,value process changes renews improves,0,"[0.14251335991122174, 0.06823987665264367, 0.7..."
25,dont hold grudges become irrelevant,0,"[0.36665984368573457, 0.47358376335450425, 0.4..."
69,feel much depressed would,1,"[0.0, 0.0, 0.9929749805355813]"


In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
tokenizer = Tokenizer()

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2021-07-30 22:57:41--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-07-30 22:57:42--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-07-30 22:57:42--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [ ]:
!unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [ ]:
train['caption'] = train['caption'].apply(str)
test['caption'] = test['caption'].apply(str)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
x_train = train.caption.to_numpy()
y_train= train.depression.to_numpy()
x_test= test.caption.to_numpy()
y_test= test.depression.to_numpy()


In [ ]:
max_seq_len = 27
tokenizer = Tokenizer()

In [ ]:
tokenizer.fit_on_texts(x_train)
word_index = tokenizer.word_index 
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size', vocab_size)

Vocabulary Size 77


In [ ]:
X = tokenizer.texts_to_sequences(x_train)

In [ ]:
X_train = pad_sequences(X,  maxlen=max_seq_len)
X = tokenizer.texts_to_sequences(x_test)
X_test = pad_sequences(X,  maxlen=max_seq_len)

In [ ]:
features_train = train.image_features.to_numpy()
features_train = np.stack(features_train)

In [ ]:
type(features_train)

numpy.ndarray

In [ ]:
type(X_train)

numpy.ndarray

In [ ]:
features_train

array([[0.52009247, 0.31945662, 0.76381135],
       [0.67053419, 0.96721353, 0.29049349],
       [0.20950223, 0.55750052, 0.5427132 ],
       [0.09117566, 0.03104957, 0.27878848],
       [0.20479229, 0.31142125, 0.76228294],
       [0.20635805, 0.25433933, 0.76237749],
       [0.31912795, 0.2524667 , 0.82059802],
       [0.18085603, 0.35166095, 0.50048545],
       [0.35736445, 0.06033329, 0.43582673],
       [0.21953918, 0.14182022, 0.75299471],
       [0.04654315, 0.05621557, 0.07385009],
       [0.59745003, 0.18199807, 0.95542219],
       [0.36866757, 0.67547368, 0.60517411],
       [0.19659416, 0.32354178, 0.5193558 ],
       [0.18166902, 0.42896013, 0.6694536 ],
       [0.50755717, 0.05962906, 0.55385647]])

In [ ]:
X_train.shape

(16, 30)

In [ ]:
# Concaternar las features de la imagen con el embedding
X_train = np.concatenate((X_train, features_train), 1)

In [ ]:
X_train.shape

(16, 33)

In [ ]:
np.stack(features_train).shape

(16, 3)

In [ ]:
features_test = test.image_features.to_numpy()
features_test = np.stack(features_test)

In [ ]:
# Concaternar las features de la imagen con el embedding
X_test = np.concatenate((X_test, features_test), 1)

In [ ]:
X_test.shape

(4, 33)

In [ ]:
print(len(tokenizer.word_index))

110


In [ ]:
y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)
print(y_train.shape)
print(y_test.shape)

(16, 1)
(4, 1)


In [ ]:
f = open("glove.6B.300d.txt", "r")

embedding_index = {}

for line in f:
    values = line.split()
    word = value = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embedding_index[word] = coefs
f.close()

print('Found word vectors', len(embedding_index))

Found word vectors 400000


In [ ]:
embedding_dim = 300

words = ['house', 'tiktok', 'dog']

for i in words:
  test = embedding_index.get(i)
  if test is not None:
    print(i)

house
dog


In [ ]:

import tensorflow as tf

embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

embedding_layer = tf.keras.layers.Embedding(vocab_size, embedding_dim, weights=[embedding_matrix],
                                           input_length=30, trainable=False)

# Modelo

In [ ]:
from keras.callbacks import ModelCheckpoint
from keras.layers import Conv1D, Bidirectional, LSTM, Input, Dropout, SpatialDropout1D, Dense

inp = Input(shape=(30,), dtype='int32')
embed_seq = embedding_layer(inp)

mod = SpatialDropout1D(0.1)(embed_seq)
mod = Bidirectional(LSTM(10, dropout=0.5, recurrent_dropout=0.5))(mod)
mod = Dropout(0.2)(mod)
output = Dense(1, activation='sigmoid')(mod)
model = keras.Model(inp, output)
model.summary()

The following Variables were used a Lambda layer's call (tf.compat.v1.nn.embedding_lookup_1), but
are not present in its tracked objects:
  <tf.Variable 'embedding_1/embeddings:0' shape=(111, 300) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Total params: 24,901
Trainable params: 24,901
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer="adam", loss='binary_crossentropy', metrics=['recall'])

In [ ]:
history = model.fit(X_train, y_train, epochs=10, batch_size=128 , validation_data=(X_test, y_test))

In [ ]:
import matplotlib.pyplot as plt

def plot_history(history):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Recall')
    plt.plot(hist['epoch'], hist['recall'], label='Train')
    plt.plot(hist['epoch'], hist['val_recall'], label = 'Test')
    plt.legend()

plot_history(history)